# Récupération des données des Eco-Compteurs (Trafic Cyclable)

## API utilisées
Nous utilisons deux endpoints du portail Montpellier 3M :

**GET `/ecocounter`**  
→ donne la liste complète des compteurs (vélo + piéton)

**GET `/ecocounter_timeseries/{ecocounterId}/attrs/intensity`**  
→ donne l’intensité du trafic par timestamp

### Imports

In [ ]:
import requests
import pandas as pd
import time


### Téléchargement et nettoyage des données relatives au trafic cycliste

In [18]:
url = "https://portail-api-data.montpellier3m.fr/ecocounter"
data = requests.get(url).json()
df = pd.DataFrame(data)
df.head()


def extract_value(col, field="value"):
    return col.apply(lambda x: x.get(field) if isinstance(x, dict) else None)

def extract_timestamp(col):
    return col.apply(
        lambda x: x["metadata"]["TimeInstant"]["value"]
        if isinstance(x, dict) and "metadata" in x and "TimeInstant" in x["metadata"]
        else None
    )

def extract_coordinates(col):
    return col.apply(
        lambda x: x["value"]["coordinates"]
        if isinstance(x, dict) and "value" in x and "coordinates" in x["value"]
        else None
    )


clean_df = pd.DataFrame()
clean_df["id"] = df["id"]

clean_df["vehicle_type"] = extract_value(df["vehicleType"])
clean_df["intensity"] = extract_value(df["intensity"])
clean_df["timestamp"] = extract_timestamp(df["intensity"])

coords = extract_coordinates(df["location"])
clean_df["lat"] = coords.apply(lambda x: x[0] if x else None)
clean_df["lon"] = coords.apply(lambda x: x[1] if x else None)

df_velo = clean_df[clean_df["vehicle_type"] == "bicycle"].reset_index(drop=True)
df_velo

,id,vehicle_type,intensity,timestamp,lat,lon
0,urn:ngsi-ld:EcoCounter:XTH19101158,bicycle,3,2025-11-26T04:00:00.000Z,43.616210,3.874408
1,urn:ngsi-ld:EcoCounter:X2H22104775,bicycle,0,2025-11-26T03:00:00.000Z,43.600100,3.877600
2,urn:ngsi-ld:EcoCounter:X2H20042632,bicycle,0,2024-02-01T02:00:00.000Z,43.590700,3.813240
3,urn:ngsi-ld:EcoCounter:X2H20063164,bicycle,0,1899-11-30T08:00:00.000Z,43.626698,3.895629
4,urn:ngsi-ld:EcoCounter:X2H19070220,bicycle,13,2025-11-25T23:00:00.000Z,43.609700,3.896940
5,urn:ngsi-ld:EcoCounter:X2H22104771,bicycle,0,2025-11-25T02:00:00.000Z,43.623300,3.908900
6,urn:ngsi-ld:EcoCounter:X2H21070350,bicycle,0,2025-01-23T02:00:00.000Z,43.539352,3.887153
7,urn:ngsi-ld:EcoCounter:X2H21070349,bicycle,0,2025-11-26T01:00:00.000Z,43.631990,3.852038
8,urn:ngsi-ld:EcoCounter:X2H21070348,bicycle,0,2010-10-10T09:00:00.000Z,43.610302,3.913589
9,urn:ngsi-ld:EcoCounter:X2H22104766,bicycle,1,2025-11-26T01:00:00.000Z,43.645200,3.822400


### Série chronologique de l'intensité du trafic cycliste

In [24]:
import pandas as pd
import json


cid = df_velo["id"].iloc[0]
url = f"https://portail-api-data.montpellier3m.fr/ecocounter_timeseries/{cid}/attrs/intensity"

ts = requests.get(url).json()

timestamps = ts["index"]
values = ts["values"]

df_timeseries = pd.DataFrame({
    "timestamp": pd.to_datetime(timestamps),
    "intensity": values,
    "ecocounter_id": ts["entityId"]
})

print(df_timeseries.info())
print(df_timeseries.describe())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   timestamp      10000 non-null  datetime64[ns, UTC]
 1   intensity      10000 non-null  float64            
 2   ecocounter_id  10000 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), object(1)
memory usage: 234.5+ KB
None
          intensity
count  10000.000000
mean      44.100200
std       63.044099
min        0.000000
25%        0.000000
50%       11.000000
75%       71.000000
max      371.000000


### Données historiques pour tous les compteurs vélo :

In [ ]:
all_dfs = []

for cid in df_velo["id"]:
    url = f"https://portail-api-data.montpellier3m.fr/ecocounter_timeseries/{cid}/attrs/intensity"
    ts = requests.get(url).json()
    
    df_ts = pd.DataFrame({
        "timestamp": pd.to_datetime(ts["index"]),
        "intensity": ts["values"],
        "ecocounter_id": cid
    })
    
    all_dfs.append(df_ts)

df_all = pd.concat(all_dfs, ignore_index=True)
print(df_all.info())


df_all.to_csv("velocounters_timeseries_all.csv", index=False)
df_all.to_parquet("velocounters_timeseries_all.parquet", index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198630 entries, 0 to 198629
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   timestamp      198630 non-null  datetime64[ns, UTC]
 1   intensity      198630 non-null  float64            
 2   ecocounter_id  198630 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), object(1)
memory usage: 4.5+ MB
None
